In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Colab_Notebooks/RIFRE-main
!pip install transformers

/content/drive/MyDrive/Colab_Notebooks/RIFRE-main


In [ ]:
import pandas as pd
import json
from pandas import json_normalize
from transformers import AutoModel, AutoTokenizer
import random

In [ ]:
with open('save_result/nyt_result.json') as data_file:
    d = json.load(data_file)

In [ ]:
w1 = json_normalize(d, 'triple_list_pred')
print(w1.to_string())

                                                                                                                              subject                                            relation                                                                                                                                   object     sub_loc     obj_loc                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 text_in
0                                                                                                                             Iceland                

In [ ]:
tk = AutoTokenizer.from_pretrained('distilbert-base-cased')

In [ ]:
def tokenize_text(tk, w1):
    sample = "[CLS]" + w1.text_in.iloc[4] +"[SEP]"
    tk_sample = tk.tokenize(sample)
    return tk_sample
tokenize_text(tk,w1)

['[CLS]',
 '(',
 '248',
 ')',
 '26',
 '##9',
 '-',
 '76',
 '##7',
 '##2',
 ',',
 'www',
 '.',
 'meadow',
 '##brook',
 '##con',
 '##co',
 '##urs',
 '.',
 'org',
 'Aug',
 '.',
 '6',
 'Formula',
 'One',
 'Grand',
 'Prix',
 ',',
 'Budapest',
 ',',
 'Hungary',
 '.',
 '[SEP]']

In [ ]:
class RDF_mapper:
    def __init__(self,df):
        self.df = df
        self.entity_dict = {}
        self.threshold = 0.2

    def assign_same_namespace(self, i , j):
        # print(self.df.iloc[i,:])
        # print(self.df.iloc[j,:])
        #Entities will be always same
        if self.df.loc[i,'rdf'] is None and self.df.loc[j,'rdf'] is None:
            if self.df.loc[i,'entity'] not in self.entity_dict.keys():
                self.df.loc[i,'rdf'] = 'www.example.org/resource/' + self.df.loc[i,'entity'] + '000'
                self.df.loc[j,'rdf'] = 'www.example.org/resource/' + self.df.loc[i,'entity'] + '000'
                self.entity_dict[self.df.loc[i,'entity']] = 1
            else:
                self.df.loc[i,'rdf'] = 'www.example.org/resource/' + self.df.loc[i,'entity'] + str(format(self.entity_dict[self.df.loc[i,'entity']],'03d'))
                self.df.loc[j,'rdf'] = 'www.example.org/resource/' + self.df.loc[i,'entity'] + str(format(self.entity_dict[self.df.loc[i,'entity']],'03d'))
                self.entity_dict[self.df.loc[i,'entity']] += 1
        elif self.df.loc[i,'rdf'] is None and self.df.loc[j,'rdf'] is not None:
            self.df.loc[i,'rdf'] = self.df.loc[j,'rdf']
            #self.entity_dict[self.df.loc[i,'entity']] = 1
        elif self.df.loc[i,'rdf'] is not None and self.df.loc[j,'rdf'] is None:
            self.df.loc[j,'rdf'] = self.df.loc[i,'rdf']
            #self.entity_dict[self.df.loc[i,'entity']] = 1
        else:
            ###REWRITE
            i_suffix = int(self.df.loc[i,'rdf'][-3:])
            j_suffix = int(self.df.loc[j,'rdf'][-3:])
            smol = min(i_suffix, j_suffix)
            self.df.loc[i,'rdf'] = 'www.example.org/resource/' + self.df.loc[i,'entity'] + str(format(smol,'03d'))
            self.df.loc[j,'rdf'] = 'www.example.org/resource/' + self.df.loc[j,'entity'] + str(format(smol,'03d'))
            #self.entity_dict[self.df.loc[i,'entity']] += 1
        return True
    
    def assign_different_namespace(self, i , j):
        if self.df.loc[i,'entity'] ==  self.df.loc[j,'entity']:
            if self.df.loc[i,'rdf'] != self.df.loc[j,'rdf']:
                return True

            if self.df.loc[i,'rdf'] is None and self.df.loc[j,'rdf'] is None:
                if self.df.loc[i,'entity'] in self.entity_dict.keys():
                    self.df.loc[i,'rdf'] = 'www.example.org/resource/' + self.df.loc[i,'entity'] + \
                    str(format(self.entity_dict[self.df.loc[i,'entity']],'03d'))
                    self.entity_dict[self.df.loc[i,'entity']] += 1
                else:
                    self.df.loc[i,'rdf'] = 'www.example.org/resource/' + self.df.loc[i,'entity'] + '000'
                    self.entity_dict[self.df.loc[i,'entity']] = 1
                if self.df.loc[j,'entity'] in self.entity_dict.keys():
                    self.df.loc[j,'rdf'] = 'www.example.org/resource/' + self.df.loc[j,'entity'] + \
                    str(format(self.entity_dict[self.df.loc[j,'entity']],'03d'))
                    self.entity_dict[self.df.loc[j,'entity']] += 1
                else:
                    self.df.loc[j,'rdf'] = 'www.example.org/resource/' + self.df.loc[j,'entity'] + '000'
                    self.entity_dict[self.df.loc[j,'entity']] = 1
            elif self.df.loc[i,'rdf'] is not None and self.df.loc[j,'rdf'] is None:
                if self.df.loc[j,'entity'] in self.entity_dict.keys():
                    self.df.loc[j,'rdf'] = 'www.example.org/resource/' + self.df.loc[j,'entity'] + \
                    str(format(self.entity_dict[self.df.loc[j,'entity']],'03d'))
                    self.entity_dict[self.df.loc[j,'entity']] += 1
                else:
                    self.df.loc[j,'rdf'] = 'www.example.org/resource/' + self.df.loc[j,'entity'] + '000'
                    self.entity_dict[self.df.loc[j,'entity']] = 1
                if self.df.loc[i,'rdf'] == self.df.loc[j,'rdf']:
                    print("CRITICAL ERROR!!!")

            elif self.df.loc[i,'rdf'] is None and self.df.loc[j,'rdf'] is not None:
                if self.df.loc[i,'entity'] in self.entity_dict.keys():
                    self.df.loc[i,'rdf'] = 'www.example.org/resource/' + self.df.loc[i,'entity'] + \
                    str(format(self.entity_dict[self.df.loc[i,'entity']],'03d'))
                    self.entity_dict[self.df.loc[i,'entity']] += 1
                else:
                    self.df.loc[i,'rdf'] = 'www.example.org/resource/' + self.df.loc[i,'entity'] + '000'
                    self.entity_dict[self.df.loc[i,'entity']] = 1
                if self.df.loc[i,'rdf'] == self.df.loc[j,'rdf']:
                    print("CRITICAL ERROR!!!")
            else:
                #Both Entities and RDF might be same
                if self.df.loc[i,'rdf'] != self.df.loc[j,'rdf']:
                    return True
                self.df.loc[i,'rdf'] = 'www.example.org/resource/' + self.df.loc[i,'entity'] + \
                    str(format(self.entity_dict[self.df.loc[i,'entity']],'03d'))
                self.entity_dict[self.df.loc[i,'entity']] += 1
        else:
            if self.df.loc[i,'rdf'] is not None and self.df.loc[j,'rdf'] is not None:
            #Since Entities are different, RDF must be different
                return True

            elif self.df.loc[i,'rdf'] is None and self.df.loc[j,'rdf'] is None:
            #RDF are not assigned
                if self.df.loc[i,'entity'] in self.entity_dict.keys():
                    self.df.loc[i,'rdf'] = 'www.example.org/resource/' + self.df.loc[i,'entity'] + \
                    str(format(self.entity_dict[self.df.loc[i,'entity']],'03d'))
                    self.entity_dict[self.df.loc[i,'entity']] += 1
                else:
                    self.df.loc[i,'rdf'] = 'www.example.org/resource/' + self.df.loc[i,'entity'] + '000'
                    self.entity_dict[self.df.loc[i,'entity']] = 1

                if self.df.loc[j,'entity'] in self.entity_dict.keys():
                    self.df.loc[j,'rdf'] = 'www.example.org/resource/' + self.df.loc[j,'entity'] + \
                    str(format(self.entity_dict[self.df.loc[j,'entity']],'03d'))
                    self.entity_dict[self.df.loc[j,'entity']] += 1
                else:
                    self.df.loc[j,'rdf'] = 'www.example.org/resource/' + self.df.loc[j,'entity'] + '000'
                    self.entity_dict[self.df.loc[j,'entity']] = 1
            
            elif self.df.loc[i,'rdf'] is None and self.df.loc[j,'rdf'] is not None:
            #Since different entity, RDF must be different
                if self.df.loc[i,'entity'] in self.entity_dict.keys():
                    self.df.loc[i,'rdf'] = 'www.example.org/resource/' + self.df.loc[i,'entity'] + \
                    str(format(self.entity_dict[self.df.loc[i,'entity']],'03d'))
                    self.entity_dict[self.df.loc[i,'entity']] += 1
                else:
                    self.df.loc[i,'rdf'] = 'www.example.org/resource/' + self.df.loc[i,'entity'] + '000'
                    self.entity_dict[self.df.loc[i,'entity']] = 1
            
            elif self.df.loc[i,'rdf'] is not None and self.df.loc[j,'rdf'] is None:
                if self.df.loc[j,'entity'] in self.entity_dict.keys():
                    self.df.loc[j,'rdf'] = 'www.example.org/resource/' + self.df.loc[j,'entity'] + \
                    str(format(self.entity_dict[self.df.loc[j,'entity']],'03d'))
                    self.entity_dict[self.df.loc[j,'entity']] += 1
                else:
                    self.df.loc[j,'rdf'] = 'www.example.org/resource/' + self.df.loc[j,'entity'] + '000'
                    self.entity_dict[self.df.loc[j,'entity']] = 1
                    if self.df.loc[i,'rdf'] == self.df.loc[j,'rdf']:
                        print("CRITICAL ERROR!!!")             
                                
        return True

    def check_distance(self):
        return round(random.uniform(0, 1), 3)

    def printer(self):
        markdict = self.entity_dict
        marklist = sorted(markdict.items(), key=lambda x:x[1])
        sortdict = dict(marklist)
        print(sortdict)
        print(self.df.to_string())
        return True

    def mapper(self):
        n = len(self.df.entity)
        for i in range(n):
            for j in range(n-i):
                #print(self.entity_dict)
                if i==n-j-1:
                    continue
                # print(i,n-j-1)
                # print(self.df.entity[i], self.df.entity[n-j-1])
                if self.df.loc[j,'entity'] != self.df.loc[n-j-1,'entity']:
                    self.assign_different_namespace(i , n-j-1)
                else:
                    #No need to check position as well as it would have been already treaated as duplicate
                    distance = self.check_distance()
                    if distance < self.threshold:
                        self.assign_different_namespace(i , n-j-1)
                    else:
                        self.assign_same_namespace(i , n-j-1)
        self.printer()


In [ ]:
df1 = w1[['subject','sub_loc', 'text_in']]
df2 = w1[['object','obj_loc', 'text_in']]
df_ent = pd.concat([df1,df2.rename(columns={'object':'subject', 'obj_loc':'sub_loc'})], ignore_index=True)
df_ent.rename(columns={'subject':'entity', 'sub_loc':'en_loc'}, inplace=True)
df_ent['entity'] = df_ent['entity'].astype(str)
df_ent['en_loc'] = df_ent['en_loc'].apply(tuple)
df_ent['text_in'] = df_ent['text_in'].astype(str)
df_ent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5216 entries, 0 to 5215
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   entity   5216 non-null   object
 1   en_loc   5216 non-null   object
 2   text_in  5216 non-null   object
dtypes: object(3)
memory usage: 122.4+ KB


In [ ]:
df_ent.head()

,entity,en_loc,text_in
0,Iceland,"(45, 46)",But that spasm of irritation by a master intim...
1,Iceland,"(45, 46)",But that spasm of irritation by a master intim...
2,Gonzalez,"(26, 27)",www.formula1.com August Aug. 1-5 National Corv...
3,Hungary,"(30, 31)","( 248 ) 269-7672 , www.meadowbrookconcours.org..."
4,Hungary,"(30, 31)","( 248 ) 269-7672 , www.meadowbrookconcours.org..."


In [ ]:
df_ent = df_ent.drop_duplicates(ignore_index=True)
df_ent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2008 entries, 0 to 2007
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   entity   2008 non-null   object
 1   en_loc   2008 non-null   object
 2   text_in  2008 non-null   object
dtypes: object(3)
memory usage: 47.2+ KB


In [ ]:
df_ent['rdf'] = None
df_ent.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,entity,en_loc,text_in,rdf
0,Iceland,"(45, 46)",But that spasm of irritation by a master intim...,None
1,Gonzalez,"(26, 27)",www.formula1.com August Aug. 1-5 National Corv...,None
2,Hungary,"(30, 31)","( 248 ) 269-7672 , www.meadowbrookconcours.org...",None
3,Budapest,"(28, 29)","( 248 ) 269-7672 , www.meadowbrookconcours.org...",None
4,Lebanon,"(11, 12)","This summer , the United States Embassy in Bei...",None


In [ ]:
RDF_mapper(df_ent).mapper()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:114

{'Vilnius': 1, 'Kandahar': 1, 'Palermo': 1, 'Rabat': 1, 'Coimbra': 1, 'Lisbon': 1, 'Allentown': 1, 'AOL': 1, 'Quito': 1, 'Varanasi': 1, 'Miskolc': 1, 'Rochester': 1, 'Island': 1, 'Basra': 1, 'Ankara': 1, 'Midland': 1, 'Saxony': 1, 'Camden': 1, 'Gedo': 1, 'Gonzaga': 1, 'Art': 1, 'Ramadi': 1, 'Florida': 1, 'Asmara': 1, "Darfur , Sudan ' s leaders in Khartoum": 1, 'Page': 1, "Damascus , Syria ' s capital , or Aleppo": 1, 'Frankfurt': 1, 'Seville': 1, 'Kut': 1, 'Yokohama': 1, 'Winnipeg': 1, 'Tallinn': 1, 'Angeles': 1, 'Belfast': 1, 'Jaipur': 1, 'Tigris': 1, 'Lome': 1, 'Nations': 1, 'Florence': 1, 'Wuhan': 1, 'Ningxiang': 1, 'Chihuahua': 1, 'Oakland': 1, 'Bridgetown': 1, 'Brasilia': 1, 'Domingo': 1, 'Astana': 1, 'Windhoek': 1, 'Beaumont': 1, 'Ukraine closer to political and economic ties with Europe , rather than Russia': 1, 'Dushanbe': 1, 'Venice': 1, 'Field': 1, 'Bratislava': 1, 'Montgomery': 1, 'Brisbane': 1, 'Lilong': 1, 'Cablevision': 1, 'Jersey': 1, 'Mumbai': 1, 'Jose': 1, 'Leipzig': 

In [ ]:
#rx.printer()

In [ ]:
#del rx

In [ ]:
df_ent.entity.value_counts()

Iraq                126
China               103
Baghdad              90
Beijing              65
Sudan                44
                   ... 
Magalhaes             1
Haight - Ashbury      1
Meri                  1
Griffith              1
Aviv                  1
Name: entity, Length: 507, dtype: int64

In [ ]:
n = 5
for i in range(n):
    for j in range(n-i):
        print(str(i)+str(n-j-1),end=' + ')
    print('\n')

04 + 03 + 02 + 01 + 00 + 

14 + 13 + 12 + 11 + 

24 + 23 + 22 + 

34 + 33 + 

44 + 



In [ ]:
x = 'jello'
x[-3:]

'llo'

In [ ]:
        # no_rows = len(self.df.entity)
        # for i in range(no_rows):
        #     for j in range(no_rows):
        #         #print(self.df.iloc[i,:],self.df.iloc[j,:])
        #         if i==j:
        #             continue
        #         if self.df.entity[i] == self.df.entity[j]:
        #             if self.df.text_in[i] == self.df.text_in[j]:
        #                 if self.df.en_loc[i] == self.df.en_loc[j]:
        #                     #assign_same_namespace(Wi, Wj)
        #                     self.assign_same_namespace(self.df.iloc[i,:],self.df.iloc[j,:], i , j)
        #                 else:
        #                     distance = self.check_distance()
        #                     if distance < self.threshold:
        #                         #assign_different_namespace(Wi,Wj)
        #                         self.assign_different_namespace(self.df.iloc[i,:],self.df.iloc[j,:], i , j)
        #                     else:
        #                         #assign_same_namespace(Wi, Wj)
        #                         self.assign_same_namespace(self.df.iloc[i,:],self.df.iloc[j,:], i , j)
        #             else:
        #                 distance = self.check_distance()
        #                 if distance >= self.threshold:
        #                     #assign_same_namespace
        #                     self.assign_same_namespace(self.df.iloc[i,:],self.df.iloc[j,:], i , j)
        #                 else:
        #                     #assign_different_namespace
        #                     self.assign_different_namespace(self.df.iloc[i,:],self.df.iloc[j,:], i , j)

In [ ]:
        # if self.df.loc[i,'rdf'] is None and self.df.loc[j,'rdf'] is None:
        #     if self.df.loc[i,'entity'] not in self.entity_dict.keys():
        #         self.df.loc[i,'rdf'] = 'www.example.org/resource/' + self.df.loc[i,'entity'] + '000'
        #         self.entity_dict[self.df.loc[i,'entity']] = 1
        #     else:
        #         self.df.loc[i,'rdf'] = 'www.example.org/resource/' + self.df.loc[i,'entity'] + str(format(self.entity_dict[self.df.loc[i,'entity']],'03d'))
        #         self.entity_dict[self.df.loc[i,'entity']] += 1     
        #     if self.df.loc[j,'entity'] not in self.entity_dict.keys(): 
        #         self.df.loc[j,'rdf'] = 'www.example.org/resource/' + self.df.loc[j,'entity'] + '000'
        #         self.entity_dict[self.df.loc[j,'entity']] = 1               
        #     else:
        #         self.df.loc[j,'rdf'] = 'www.example.org/resource/' + self.df.loc[j,'entity'] + str(format(self.entity_dict[self.df.loc[j,'entity']],'03d'))
        #         self.entity_dict[self.df.loc[j,'entity']] += 1 

        # elif self.df.loc[i,'rdf'] is None and self.df.loc[j,'rdf'] is not None:

        #     self.df.loc[i,'rdf'] = 'www.example.org/resource/' + self.df.loc[i,'entity'] + '000'
        #     self.entity_dict[self.df.loc[i,'entity']] = 1

        # elif self.df.loc[i,'rdf'] is not None and self.df.loc[j,'rdf'] is not None:
        #     if self.df.loc[i,'rdf'] == self.df.loc[j,'rdf']:
        #         self.df.loc[i,'rdf'] = 'www.example.org/resource/' + self.df.loc[i,'entity'] + str(format(self.entity_dict[self.df.loc[i,'entity']],'03d'))
        #         self.entity_dict[self.df.loc[i,'entity']] += 1
        #         self.df.loc[j,'rdf'] = 'www.example.org/resource/' + self.df.iloc[j,:].entity + str(format(self.entity_dict[self.df.iloc[j,:].entity],'03d'))
        #         self.entity_dict[self.df.loc[i,'entity']] += 1